In [1]:
!pip install opencv-python-headless 
!pip install mediapipe 
!pip install --upgrade numexpr 

import cv2
import mediapipe as mp
import numpy as np
import time
from scipy import signal
import pandas as pd
import re
import os
import gc  

# Define the path for storing the DataFrame
df_path = "processed_videos.csv"

# Load the DataFrame if it exists; otherwise, create an empty one
if os.path.exists(df_path):
    df = pd.read_csv(df_path)
    print(f"Loaded existing DataFrame with {len(df)} entries.")
else:
    df = pd.DataFrame(columns=["ID", "Phase", "HRV"])
    print("Created a new empty DataFrame.")

# List to track files that encountered errors
error_files = []

# Directory containing video files
video_directory = "videos"

# Define the heart rate estimation class
class HeartRateEstimator:
    def __init__(self, buffer_size=150):
        self.buffer_size = buffer_size
        self.data_buffer = []  # Stores green channel intensity over time
        self.times = []  # Stores timestamps of the frames
        self.t0 = time.time()  # Start time for frame timestamps

    def extract_green_channel(self, frame):
        green_mean = np.mean(frame[:, :, 1])
        return green_mean

    def process_frame(self, frame):
        green_value = self.extract_green_channel(frame)
        self.times.append(time.time() - self.t0)
        self.data_buffer.append(green_value)

        # Only start processing when buffer is filled
        if len(self.data_buffer) > self.buffer_size:
            # Keep only the last `buffer_size` elements
            self.data_buffer = self.data_buffer[-self.buffer_size:]
            self.times = self.times[-self.buffer_size:]

            # Estimate heart rate using the data buffer
            bpm = self.estimate_heart_rate()
            return bpm

        return None

    def estimate_heart_rate(self):
        clean_data_buffer = np.array(self.data_buffer)
        clean_data_buffer = clean_data_buffer[np.isfinite(clean_data_buffer)]
        
        if len(clean_data_buffer) < 2:
            print("Not enough valid data points to estimate heart rate.")
            return 0
        
        L = len(clean_data_buffer)
        fps = float(L) / (self.times[-1] - self.times[0])
        detrended = signal.detrend(clean_data_buffer)
        peaks, _ = signal.find_peaks(detrended, distance=fps * 0.6)

        peak_times = np.array(self.times)[peaks]
        rr_intervals = np.diff(peak_times)

        if len(rr_intervals) > 0:
            avg_rr_interval = np.mean(rr_intervals)
            bpm = 60 / avg_rr_interval
        else:
            bpm = 0

        return bpm

# Define the face detection class
class FaceDetection:
    def __init__(self):
        self.face_detection = mp.solutions.face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

    def detect_face(self, frame):
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_detection.process(frame_rgb)

        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = frame.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                face_frame = frame[y:y + h, x:x + w]
                return face_frame

        return None 

# Function to calculate RMSSD
def calculate_rmssd(bpm_series):
    try:
        hr_variability = np.diff(bpm_series)
        squared_diffs = np.square(hr_variability)
        mean_squared_diffs = np.mean(squared_diffs)
        rmssd = np.sqrt(mean_squared_diffs)
        return rmssd
    except TypeError as e:
        print(f"Error calculating RMSSD: {e}")
        return None

# Function to add an entry to the DataFrame
def add_entry(ID, Phase, rmssd):
    global df
    # Check if this entry already exists in the DataFrame
    if not ((df["ID"] == ID) & (df["Phase"] == Phase)).any():
        new_entry = pd.DataFrame([{"ID": ID, "Phase": Phase, "HRV": rmssd}])
        df = pd.concat([df, new_entry], ignore_index=True)
        print(f"Added: {new_entry.iloc[0].to_dict()}")
    else:
        print(f"Entry with ID {ID} and Phase {Phase} already exists. Skipping.")

# Process each video file in the directory in batches
error_files = []
for idx, filename in enumerate(os.listdir(video_directory)):
    if filename.endswith(".MOV"):
        # Parse ID and Phase from the filename using regex that matches both "-" and "_" separators
        match = re.match(r"(\d+)[-_](\d+)\.MOV", filename)
        if match:
            ID = int(match.group(1))
            Phase = int(match.group(2))

            # Check if this file's data is already in the DataFrame to avoid re-processing
            if ((df["ID"] == ID) & (df["Phase"] == Phase)).any():
                print(f"Skipping {filename}: ID {ID} and Phase {Phase} already in DataFrame.")
                continue  # Skip this file if already processed

            file_path = os.path.join(video_directory, filename)
            try:
                if os.path.exists(file_path):
                    cap = cv2.VideoCapture(file_path)
                    if not cap.isOpened():
                        # Log the filename that causes the "moov atom not found" error
                        print(f"Error: Could not open video '{filename}' (moov atom not found).")
                        error_files.append(f"{filename} (moov atom not found)")
                        continue
                    
                    estimator = HeartRateEstimator()
                    face_detector = FaceDetection()
                    bpm_series = []

                    while cap.isOpened():
                        ret, frame = cap.read()
                        if not ret:
                            break

                        face_frame = face_detector.detect_face(frame)
                        if face_frame is not None:
                            bpm = estimator.process_frame(face_frame)
                            if bpm is not None:
                                bpm_series.append(bpm)

                    cap.release()

                    if len(bpm_series) > 1:
                        rmssd = calculate_rmssd(bpm_series)
                        if rmssd is not None:
                            print(f"RMSSD: {rmssd:.2f}")
                            add_entry(ID, Phase, rmssd)
                        else:
                            print("Unable to calculate RMSSD.")
                            error_files.append(f"{filename} (RMSSD calculation failed)")
                    else:
                        print("Not enough BPM data to calculate RMSSD.")
                        error_files.append(f"{filename} (insufficient BPM data)")
                else:
                    print(f"Video file '{file_path}' does not exist.")
                    error_files.append(f"{filename} (file not found)")
            
            except Exception as e:
                print(f"An error occurred while processing '{filename}': {e}")
                error_files.append(f"{filename} (processing error: {e})")
        
            # Clear memory by deleting large objects after each file
            try:
                del cap, estimator, face_detector, bpm_series
            except NameError:
                pass  # If any variable wasn't defined, just skip deleting it.
            
            gc.collect()  # Run garbage collection to free memory

            # Save periodically after every 10 files
            if (idx + 1) % 10 == 0:
                df.to_csv(df_path, index=False)
                print(f"Data saved to '{df_path}' after processing {idx + 1} files.")

# Final save of the DataFrame
df.to_csv(df_path, index=False)
print(f"Final data saved to '{df_path}'.")

# Display the DataFrame and list of error files
print(df)
if error_files:
    print("\nFiles that encountered errors and were not added to the DataFrame:")
    for error_file in error_files:
        print(f"- {error_file}")
else:
    print("\nAll files processed successfully.")


2024-11-11 08:53:09.041048: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loaded existing DataFrame with 621 entries.
Error: Could not open video '48_01.MOV' (moov atom not found).
Skipping 168_02.MOV: ID 168 and Phase 2 already in DataFrame.
Skipping 246_01.MOV: ID 246 and Phase 1 already in DataFrame.
Skipping 175_01.MOV: ID 175 and Phase 1 already in DataFrame.
Skipping 135_01.MOV: ID 135 and Phase 1 already in DataFrame.
Skipping 312_02.MOV: ID 312 and Phase 2 already in DataFrame.
Skipping 242_02.MOV: ID 242 and Phase 2 already in DataFrame.
Skipping 189_02.MOV: ID 189 and Phase 2 already in DataFrame.
Skipping 321_01.MOV: ID 321 and Phase 1 already in DataFrame.
Skipping 132_01.MOV: ID 132 and Phase 1 already in DataFrame.
Skipping 100_01.MOV: ID 100 and Phase 1 already in DataFrame.
Skipping 315_02.MOV: ID 315 and Phase 2 already in DataFrame.
Error: Could not open video '62_01.MOV' (moov atom not found).
Skipping 255_01.MOV: ID 255 and Phase 1 already in DataFrame.
Skipping 150_01.MOV: ID 150 and Phase 1 already in DataFrame.
Skipping 108_02.MOV: ID 

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d136043a40] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d136044f00] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d136045500] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d1360458c0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d136445f00] moov atom not found


Error: Could not open video '30_01.MOV' (moov atom not found).
Skipping 120_02.MOV: ID 120 and Phase 2 already in DataFrame.
Skipping 249_02.MOV: ID 249 and Phase 2 already in DataFrame.
Skipping 89_02.MOV: ID 89 and Phase 2 already in DataFrame.
Skipping 141_02.MOV: ID 141 and Phase 2 already in DataFrame.
Skipping 217_02.MOV: ID 217 and Phase 2 already in DataFrame.
Skipping 280_01.MOV: ID 280 and Phase 1 already in DataFrame.
Skipping 111_01.MOV: ID 111 and Phase 1 already in DataFrame.
Error: Could not open video '77_01.MOV' (moov atom not found).
Skipping 88_02.MOV: ID 88 and Phase 2 already in DataFrame.
Skipping 38_01.MOV: ID 38 and Phase 1 already in DataFrame.
Skipping 80_01.MOV: ID 80 and Phase 1 already in DataFrame.
Skipping 22-02.MOV: ID 22 and Phase 2 already in DataFrame.
Skipping 27-02.MOV: ID 27 and Phase 2 already in DataFrame.
Skipping 114_01.MOV: ID 114 and Phase 1 already in DataFrame.
Skipping 183_01.MOV: ID 183 and Phase 1 already in DataFrame.
Skipping 179_03.MO

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d136446300] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d136045b80] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d1364465c0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d136446900] moov atom not found


...

RMSSD: 1.59
Added: {'ID': 223.0, 'Phase': 1.0, 'HRV': 1.5855058446086912}


W0000 00:00:1731318293.326065   42770 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


RMSSD: 1.47
Added: {'ID': 103.0, 'Phase': 1.0, 'HRV': 1.4680536265760973}
Error: Could not open video '40_01.MOV' (moov atom not found).


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55d1370fea00] moov atom not found


Final data saved to 'processed_videos.csv'.
      ID  Phase       HRV
0    168      2  1.347618
1    246      1  1.571045
2    175      1  1.601651
3    135      1  1.843167
4    312      2  1.648120
..   ...    ...       ...
630  117      2  1.625016
631  284      2  1.545879
632  278      1  1.242224
633  223      1  1.585506
634  103      1  1.468054

[635 rows x 3 columns]

Files that encountered errors and were not added to the DataFrame:
- 48_01.MOV (moov atom not found)
- 62_01.MOV (moov atom not found)
- 30_01.MOV (moov atom not found)
- 77_01.MOV (moov atom not found)
- 51_03.MOV (moov atom not found)
- 77_03.MOV (moov atom not found)
- 31_02.MOV (moov atom not found)
- 60_02.MOV (moov atom not found)
- 56_01.MOV (moov atom not found)
- 59_02.MOV (moov atom not found)
- 82_01.MOV (processing error: `distance` must be greater or equal to 1)
- 29_01.MOV (moov atom not found)
- 79_02.MOV (moov atom not found)
- 53_02.MOV (moov atom not found)
- 21-01.MOV (moov atom not found)
- 4